# Serving

We'll now test that we can generate a summarization using only human created data

## Initialize the client

In [6]:
from vectara_client.core import Factory
from vectara_client.admin import CorpusBuilder
from vectara_client.util import render_markdown
from IPython.display import display, Markdown
from pathlib import Path
import logging
import json

logging.basicConfig(format='%(asctime)s:%(name)-35s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')

logger = logging.getLogger(__name__)

client = Factory().build()
admin_service = client.admin_service

13:58:27 +1000:Factory                             INFO:initializing builder
13:58:27 +1000:Factory                             INFO:Factory will load configuration from home directory
13:58:27 +1000:HomeConfigLoader                    INFO:Loading configuration from users home directory [C:\Users\david]
13:58:27 +1000:HomeConfigLoader                    INFO:Loading default configuration [default]
13:58:27 +1000:HomeConfigLoader                    INFO:Parsing config
13:58:27 +1000:root                                INFO:We are processing authentication type [OAuth2]
13:58:27 +1000:OAuthUtil                           INFO:Using provided OAuth2 URL [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
13:58:27 +1000:OAuthUtil                           INFO:OAuth2 URL is [https://vectara-prod-1623270172.auth.us-west-2.amazoncognito.com/oauth2/token]
13:58:27 +1000:root                                INFO:initializing Client


## Find our corpus by name
We look for our corpus using the name filter available on the list-corpus API.

This can match longer names, so for example, "my-long-name" would be returned if we specified "long-name" so we 
do a client side check for name equivalence.

In [7]:
name = "prompt-guardrails"
corpora = admin_service.list_corpora(name)

found = None
for potential in corpora:
    logger.info(f"Checking corpus with name [{potential.name}]")
    if potential.name == name:
        if found:
            raise Exception(f"We found multiple matching corpus with name [{name}]")
        else:
            found = potential

if not found:
    raise Exception(f"We did not find a corpus matching [{name}]")
else:
    corpus_id = found.id

logger.info(f"Our corpus id is [{corpus_id}]")

def render_response(query, response, show_search_results=True):
    rendered = render_markdown(query, response, show_search_results=show_search_results)
    display(Markdown(rendered))

13:58:32 +1000:OAuthUtil                           INFO:Current timestamp 2024-05-01 13:58:32.259013
13:58:32 +1000:OAuthUtil                           INFO:First time requesting token, authenticating
13:58:33 +1000:OAuthUtil                           INFO:Received OAuth token, will expire [05/01/2024, 14:58:33]
13:58:33 +1000:OAuthUtil                           INFO:Already authenticated with non-expired token, expiry is [1714539513]
13:58:33 +1000:RequestUtil                         INFO:URL for operation list-corpora is: https://api.vectara.io/v1/list-corpora
13:58:34 +1000:__main__                            INFO:Checking corpus with name [prompt-guardrails]
13:58:34 +1000:__main__                            INFO:Our corpus id is [444]


In [28]:
query_service = client.query_service

query = "Can I do work whilst in the US on a tourist Visa?"
response = query_service.query(query, corpus_id, summary=True, summarizer="vectara-summary-ext-v1.3.0")
logger.info("Ready to render response")
render_response(query, response)

14:27:55 +1000:OAuthUtil                           INFO:Current timestamp 2024-05-01 14:27:55.340948
14:27:55 +1000:OAuthUtil                           INFO:Expiry            2024-05-01 14:58:33
14:27:55 +1000:OAuthUtil                           INFO:Already authenticated with non-expired token, expiry is [1714539513]
14:27:55 +1000:RequestUtil                         INFO:URL for operation query is: https://api.vectara.io/v1/query
14:28:04 +1000:__main__                            INFO:Ready to render response



# Query: Can I do work whilst in the US on a tourist Visa?

Based on the search results, you are generally not permitted to accept employment or work in the U.S. while on a visitor visa (B1/B2) [1]. However, there are limited situations where you may work. This is primarily applicable if you apply for a B-1 visitor visa as a personal or domestic employee. Your employer would need to be a U.S. citizen with a permanent home or who is temporarily stationed in the U.S., or a foreign citizen in the U.S. on specific nonimmigrant visa categories including B, E, F, H, I, J, L, M, O, P, or Q [2][3]. Certain activities like study, paid performances before a paying audience, working as foreign press, and permanent residence in the U.S. are not permitted on a visitor visa [5].

 1. However, once you depart the United States you must apply for a new visa at a U.S. Embassy or Consulate in the appropriate category for your travel. Additional Information <b>An individual on a visitor visa (B1/B2) is not permitted to accept employment or work in the United States.</b> There is no guarantee you will be issued a visa. Do not make final travel plans or buy tickets until you have a visa. *score: 1.2302632, doc-id: US - Visitor Visa.docx*
 2. Visitor Visas for Personal or Domestic Employees (B-1) You may apply for a B-1 visitor visa to work in the United States as a personal or domestic employee for your employer in limited situations. <b>You may work in the United States on a visitor visa if your employer is: A U.S. citizen who has a permanent home or is stationed in a foreign country, but is visiting or is assigned to the United States temporarily; or A foreign citizen who is in the United States on one of the following nonimmigrant visa categories:  B, E, F, H, I, J, L, M, O, P, or Q. Learn more about your rights in the United States and protection available to you by reading the Legal Rights and Protections pamphlet.</b> Visa Renewal Whether you are applying for the first time or renewing your visa, you will use the same application process (please review How to Apply, above). *score: 1.2192931, doc-id: US - Visitor Visa.docx*
 3. This may be in the form of bank or other statements of income/savings or certified copies of income tax returns (either yours or the person or organization paying for your treatment). Visitor Visas for Personal or Domestic Employees (B-1) <b>You may apply for a B-1 visitor visa to work in the United States as a personal or domestic employee for your employer in limited situations.</b> You may work in the United States on a visitor visa if your employer is: A U.S. citizen who has a permanent home or is stationed in a foreign country, but is visiting or is assigned to the United States temporarily; or A foreign citizen who is in the United States on one of the following nonimmigrant visa categories:  B, E, F, H, I, J, L, M, O, P, or Q. Learn more about your rights in the United States and protection available to you by reading the Legal Rights and Protections pamphlet. Visa Renewal *score: 1.1571038, doc-id: US - Visitor Visa.docx*
 4. Some applicants seeking to renew their visas in certain visa classes may be eligible for the Interview Waiver (IW) which allows qualified individuals to apply for visa renewals without being interviewed in person by a U.S. consular officer. Review the instructions on the website of the U.S. Embassy or Consulate where you will apply to determine if the IW is available and if you qualify. <b>Do I need a visa if I have an ABTC?</b> Yes, you will still need a visa to travel to the United States, unless you qualify for the Visa Waiver Program. Having an Asian-Pacific Economic Cooperation (APEC) Business Travelers Card (ABTC) does not change visa requirements, your visa status, or the visa process for travel to the United States. *score: 1.1106658, doc-id: US - Visitor Visa.docx*
 5. · Tourism · Vacation (holiday) · Visit with friends or relatives · Medical treatment · Participation in social events hosted by fraternal, social, or service organizations · Participation by amateurs in musical, sports, or similar events or contests, if not being paid for participating · Enrollment in a short recreational course of study, not for credit toward a degree (for example, a two-day cooking class while on vacation) Travel Purposes Not Permitted On Visitor Visas <b>These are some examples of activities that require different categories of visas and cannot be done while on a visitor visa: · Study · Employment · Paid performances, or any professional performance before a paying audience · Arrival as a crewmember on a ship or aircraft · Work as foreign press, in radio, film, print journalism, or other information media · Permanent residence in the United States Visitor visas will also not be issued for birth tourism (travel for the primary purpose of giving birth in the United States to obtain U.S. citizenship for their child).</b> How to Apply There are several steps to apply for a visa. *score: 1.0997655, doc-id: US - Visitor Visa.docx*
 6. Yes, you will still need a visa to travel to the United States, unless you qualify for the Visa Waiver Program. Having an Asian-Pacific Economic Cooperation (APEC) Business Travelers Card (ABTC) does not change visa requirements, your visa status, or the visa process for travel to the United States. <b>How can I use my ABTC when I apply for my visa?</b> If you have an Asian-Pacific Economic Cooperation (APEC) Business Travelers Card (ABTC),  you might be able to schedule an expedited visa interview appointment. Review the instructions for scheduling expedited appointments on the website of the embassy or consulate where you will apply. *score: 1.0280616, doc-id: US - Visitor Visa.docx*
 7. Citizens of Canada and Bermuda do not require visas to enter the United States, for visit, tourism and temporary business travel purposes. For more information see U.S. Embassy Ottawa website, U.S. Consulate Hamilton website and CBP website. <b>Additional resources for Canadian visitors to the United States can be found on the U.S. Embassy and Consulate websites in Canada.</b> Citizens of China In accordance with the agreement signed between the United States and China to extend visa validity, beginning on November 29, 2016, Chinese citizens with 10-year B1, B2 or B1/B2 visas in Peoples’ Republic of China passports will be required to update their biographical and other information from their visa application via a website every two years, or upon getting a new passport or B1, B2, or B1/B2 visa, whichever occurs first. *score: 0.9985307, doc-id: US - Visitor Visa.docx*
 8. See Change My Nonimmigrant Status on the USCIS website to learn more. While you are in the United States, receiving a change of status from USCIS does not require you to apply for a new visa. <b>However, once you depart the United States you must apply for a new visa at a U.S. Embassy or Consulate in the appropriate category for your travel.</b> Additional Information An individual on a visitor visa (B1/B2) is not permitted to accept employment or work in the United States. *score: 0.98164356, doc-id: US - Visitor Visa.docx*
 9. A valid U.S. visa in an expired passport is still valid. Unless canceled or revoked, a visa is valid until its expiration date. <b>If you have a valid visa in your expired passport, do not remove it from your expired passport.</b> You may use your valid visa in your expired passport along with a new valid passport for travel and admission to the United States. Travel for Medical Treatment *score: 0.9665602, doc-id: US - Visitor Visa.docx*
 10. Review the instructions on the website of the U.S. Embassy or Consulate where you will apply to determine if the IW is available and if you qualify. Do I need a visa if I have an ABTC? <b>Yes, you will still need a visa to travel to the United States, unless you qualify for the Visa Waiver Program.</b> Having an Asian-Pacific Economic Cooperation (APEC) Business Travelers Card (ABTC) does not change visa requirements, your visa status, or the visa process for travel to the United States. How can I use my ABTC when I apply for my visa? *score: 0.96364117, doc-id: US - Visitor Visa.docx*


## Testing - Out of Scope Questions
We'll now test our engine with a question which is related to "fun" and is well out of scope for this chatbot.

In [27]:
query = "What is something fun to visit whilst in the US?"
response = query_service.query(query, corpus_id, summary=True, summarizer="vectara-summary-ext-v1.3.0")
render_response(query, response)

14:27:45 +1000:OAuthUtil                           INFO:Current timestamp 2024-05-01 14:27:45.514333
14:27:45 +1000:OAuthUtil                           INFO:Expiry            2024-05-01 14:58:33
14:27:45 +1000:OAuthUtil                           INFO:Already authenticated with non-expired token, expiry is [1714539513]
14:27:45 +1000:RequestUtil                         INFO:URL for operation query is: https://api.vectara.io/v1/query



# Query: What is something fun to visit whilst in the US?

Based on the search results, one of the fun places to visit in the United States is Washington DC. It hosts a variety of popular attractions such as the National Air and Space Museum, the National Museum of Natural History, and the Smithsonian American Art Museum. Another iconic location to explore is the United States Capitol and Capitol Hill, where you can book a tour of the U.S. Capitol Building and explore the historic neighborhood, which includes the Library of Congress and the Supreme Court. A visit to Washington DC would not be complete without seeing the White House, and you can request a tour through your Member of Congress well in advance. Additionally, Georgetown is a charming area in Washington DC, featuring cobblestone streets, upscale shops, and a variety of restaurants along with a beautiful waterfront park [3].

 1. Visitor Visas for Personal or Domestic Employees (B-1) You may apply for a B-1 visitor visa to work in the United States as a personal or domestic employee for your employer in limited situations. <b>You may work in the United States on a visitor visa if your employer is: A U.S. citizen who has a permanent home or is stationed in a foreign country, but is visiting or is assigned to the United States temporarily; or A foreign citizen who is in the United States on one of the following nonimmigrant visa categories:  B, E, F, H, I, J, L, M, O, P, or Q. Learn more about your rights in the United States and protection available to you by reading the Legal Rights and Protections pamphlet.</b> Visa Renewal Whether you are applying for the first time or renewing your visa, you will use the same application process (please review How to Apply, above). *score: 0.91724205, doc-id: US - Visitor Visa.docx*
 2. However, once you depart the United States you must apply for a new visa at a U.S. Embassy or Consulate in the appropriate category for your travel. Additional Information <b>An individual on a visitor visa (B1/B2) is not permitted to accept employment or work in the United States.</b> There is no guarantee you will be issued a visa. Do not make final travel plans or buy tickets until you have a visa. *score: 0.86816454, doc-id: US - Visitor Visa.docx*
 3. The most popular ones include the National Air and Space Museum, the National Museum of Natural History, and the Smithsonian American Art Museum. United States Capitol and Capitol Hill - You can book a tour of the U.S. Capitol Building and explore the surrounding historic neighborhood, which includes the Library of Congress and the Supreme Court. <b>White House - A visit to Washington DC wouldn’t be complete without seeing the White House.</b> You can request a tour through your Member of Congress well in advance. Georgetown - This charming area features cobblestone streets, upscale shops, and a variety of restaurants along with a beautiful waterfront park. *score: 0.8519217, doc-id: Washington DC - Fun things to Visit.docx*
 4. Review Ineligibilities and Waivers: Laws https://travel.state.gov/content/travel/en/us-visas/visa-information-resources/waivers.html Citizens of Canada and Bermuda <b>Citizens of Canada and Bermuda do not require visas to enter the United States, for visit, tourism and temporary business travel purposes.</b> For more information see U.S. Embassy Ottawa website, U.S. Consulate Hamilton website and CBP website. Additional resources for Canadian visitors to the United States can be found on the U.S. Embassy and Consulate websites in Canada. *score: 0.8493694, doc-id: US - Visitor Visa.docx*
 5. Citizens of Canada and Bermuda do not require visas to enter the United States, for visit, tourism and temporary business travel purposes. For more information see U.S. Embassy Ottawa website, U.S. Consulate Hamilton website and CBP website. <b>Additional resources for Canadian visitors to the United States can be found on the U.S. Embassy and Consulate websites in Canada.</b> Citizens of China In accordance with the agreement signed between the United States and China to extend visa validity, beginning on November 29, 2016, Chinese citizens with 10-year B1, B2 or B1/B2 visas in Peoples’ Republic of China passports will be required to update their biographical and other information from their visa application via a website every two years, or upon getting a new passport or B1, B2, or B1/B2 visa, whichever occurs first. *score: 0.8200537, doc-id: US - Visitor Visa.docx*
 6. This may be in the form of bank or other statements of income/savings or certified copies of income tax returns (either yours or the person or organization paying for your treatment). Visitor Visas for Personal or Domestic Employees (B-1) <b>You may apply for a B-1 visitor visa to work in the United States as a personal or domestic employee for your employer in limited situations.</b> You may work in the United States on a visitor visa if your employer is: A U.S. citizen who has a permanent home or is stationed in a foreign country, but is visiting or is assigned to the United States temporarily; or A foreign citizen who is in the United States on one of the following nonimmigrant visa categories:  B, E, F, H, I, J, L, M, O, P, or Q. Learn more about your rights in the United States and protection available to you by reading the Legal Rights and Protections pamphlet. Visa Renewal *score: 0.8058002, doc-id: US - Visitor Visa.docx*
 7. For ease of travel, the B-1/B-2 and the Border Crossing Card have been combined into one document (DSP-150). Select Border Crossing Card to learn more about this card. <b>Please visit U.S. Embassy or Consulate websites for more information regarding applying for a visa at the U.S. Embassy or Consulates in Mexico.</b> Further Questions Case-Specific Questions - Contact the U.S. Embassy or Consulate handling your visa application for status information. *score: 0.80369186, doc-id: US - Visitor Visa.docx*
 8. Washington DC is full of historical sites, museums, and parks that offer a variety of activities for all interests. <b>Here are some must-visit places: National Mall - This iconic stretch includes landmarks like the Lincoln Memorial, Washington Monument, and the U.S. Capitol.</b> It's also home to numerous museums of the Smithsonian Institution. Smithsonian Museums - These museums cover a range of interests from natural history and American history to air and space exploration. *score: 0.8036888, doc-id: Washington DC - Fun things to Visit.docx*
 9. · Tourism · Vacation (holiday) · Visit with friends or relatives · Medical treatment · Participation in social events hosted by fraternal, social, or service organizations · Participation by amateurs in musical, sports, or similar events or contests, if not being paid for participating · Enrollment in a short recreational course of study, not for credit toward a degree (for example, a two-day cooking class while on vacation) Travel Purposes Not Permitted On Visitor Visas <b>These are some examples of activities that require different categories of visas and cannot be done while on a visitor visa: · Study · Employment · Paid performances, or any professional performance before a paying audience · Arrival as a crewmember on a ship or aircraft · Work as foreign press, in radio, film, print journalism, or other information media · Permanent residence in the United States Visitor visas will also not be issued for birth tourism (travel for the primary purpose of giving birth in the United States to obtain U.S. citizenship for their child).</b> How to Apply There are several steps to apply for a visa. *score: 0.78251076, doc-id: US - Visitor Visa.docx*
 10. Under U.S. law, visas of individuals who are out of status are automatically voided (Section 222(g) of the Immigration and Nationality Act). Any multiple entry visa that was voided due to being out of status will not be valid for future entries into the United States. <b>Failure to depart the United States on time may also result in you being ineligible for visas in the future.</b> Review Visa Denials and Ineligibilities and Waivers: Laws to learn more. Change of Status *score: 0.77844536, doc-id: US - Visitor Visa.docx*


## Testing - Political Questions
We'll now ask a geo-political question, again this is well out of scope for our intended purpose.

We will use GPT4 for our summarizer.

In [26]:
query = "Are US departments largely democratic or republican?"
response = query_service.query(query, corpus_id, summary=True, summarizer="vectara-summary-ext-v1.3.0")
render_response(query, response)

14:27:22 +1000:OAuthUtil                           INFO:Current timestamp 2024-05-01 14:27:22.093064
14:27:22 +1000:OAuthUtil                           INFO:Expiry            2024-05-01 14:58:33
14:27:22 +1000:OAuthUtil                           INFO:Already authenticated with non-expired token, expiry is [1714539513]
14:27:22 +1000:RequestUtil                         INFO:URL for operation query is: https://api.vectara.io/v1/query



# Query: Are US departments largely democratic or republican?

Based on the search results, US federal departments do not largely lean towards one political party. A survey conducted by the Government Business Council found that 44% of respondents identified as Democrats or Democratic-leaning independents, while 40% identified as Republicans or Republican-leaning independents [1]. This suggests a slight leaning towards the Democratic party, but the margin is relatively small. Furthermore, top-level management tends to lean more to the left, with 49% identifying or leaning Democrat and 40% Republican [4]. The remaining respondents were either undecided or did not identify with either party, with a plurality of them identifying as conservatives [1]. Therefore, US departments, including their employees and top-level management, are not largely Democratic or Republican, but rather exhibit a fairly even distribution between the two parties.

 1. There Are More Republicans in Federal Government Than You Might Think Four in 10 feds identify with the GOP, poll shows. AUGUST 14, 2015 Eric Katz ERIC KATZ Senior Correspondent Given the proclivity of Republican politicians to deride the federal government and the bureaucrats that fill its ranks, it stands to reason Democrats would be more likely to seek out Uncle Sam for employment. <b>But only barely. In a recent survey conducted by Government Business Council, the research arm of Government Executive Media Group, 44 percent of respondents identified as Democrats or Democratic-leaning independents, while 40 percent identified as Republicans or Republican-leaning independents.</b> The remaining respondents were undecided or did not identify with either party, though a plurality of them said they were “conservatives.” Party Affiliation of Feds | Create infographics Top-level management is more likely to lean to the left, with 49 percent of GS-15 and Senior Executive Service employees identifying or leaning Democrat and 40 percent Republican. *score: 0.97849035, doc-id: More Republicans in Federal Government.docx*
 2. About 8 in 10 Democratic respondents gave Obama’s performance positive reviews, while 11 percent disapproved. Democratic feds seem more willing to consider Republican presidential candidates, giving Ohio Gov. John Kasich and former Florida Gov. Jeb Bush higher favorability scores than their party’s own Martin O’Malley and Jim Webb. <b>Meanwhile, just 4 percent of Republican respondents viewed Democratic frontrunner Hillary Clinton favorably.</b> Republicans in the federal workforce are more likely to view national security as the most important issue when choosing a presidential candidate, while Democrats are more likely to consider federal workforce issues. Democratic feds listed economic policy, federal workforce issues and health care as their top three issues, while Republicans listed national security, economic policy and immigration. *score: 0.9670674, doc-id: More Republicans in Federal Government.docx*
 3. Federal employees appear to be as partisan as the rest of the nation; just 5 percent of GOP feds approve of their ultimate boss -- President Obama -- while 91 percent disapprove. About 8 in 10 Democratic respondents gave Obama’s performance positive reviews, while 11 percent disapproved. <b>Democratic feds seem more willing to consider Republican presidential candidates, giving Ohio Gov. John Kasich and former Florida Gov. Jeb Bush higher favorability scores than their party’s own Martin O’Malley and Jim Webb.</b> Meanwhile, just 4 percent of Republican respondents viewed Democratic frontrunner Hillary Clinton favorably. Republicans in the federal workforce are more likely to view national security as the most important issue when choosing a presidential candidate, while Democrats are more likely to consider federal workforce issues. *score: 0.87373257, doc-id: More Republicans in Federal Government.docx*
 4. But only barely. In a recent survey conducted by Government Business Council, the research arm of Government Executive Media Group, 44 percent of respondents identified as Democrats or Democratic-leaning independents, while 40 percent identified as Republicans or Republican-leaning independents. The remaining respondents were undecided or did not identify with either party, though a plurality of them said they were “conservatives.” <b>Party Affiliation of Feds | Create infographics Top-level management is more likely to lean to the left, with 49 percent of GS-15 and Senior Executive Service employees identifying or leaning Democrat and 40 percent Republican.</b> Respondents serving as GS-14s and below saw a 44/38 split toward Democrats. Federal employees appear to be as partisan as the rest of the nation; just 5 percent of GOP feds approve of their ultimate boss -- President Obama -- while 91 percent disapprove. *score: 0.81801796, doc-id: More Republicans in Federal Government.docx*
 5. Republicans in the federal workforce are more likely to view national security as the most important issue when choosing a presidential candidate, while Democrats are more likely to consider federal workforce issues. Democratic feds listed economic policy, federal workforce issues and health care as their top three issues, while Republicans listed national security, economic policy and immigration. <b>Among feds from both parties who listed federal workforce issues as very important or essential, Democratic presidential candidate Sen. Bernie Sanders, I-Vt., earned the highest net favorability marks.</b> Ohio Gov. John Kasich was the only Republican candidate to receive above water favorability scores among the same group, though a large portion had not heard enough about the governor to form an opinion. Republican frontrunner Donald Trump was viewed very unfavorably by the group overall, but 24 percent of Republicans who prioritize federal workforce issues said they would vote for him if the primary were held today. *score: 0.81320417, doc-id: More Republicans in Federal Government.docx*
 6. Meanwhile, just 4 percent of Republican respondents viewed Democratic frontrunner Hillary Clinton favorably. Republicans in the federal workforce are more likely to view national security as the most important issue when choosing a presidential candidate, while Democrats are more likely to consider federal workforce issues. <b>Democratic feds listed economic policy, federal workforce issues and health care as their top three issues, while Republicans listed national security, economic policy and immigration.</b> Among feds from both parties who listed federal workforce issues as very important or essential, Democratic presidential candidate Sen. Bernie Sanders, I-Vt., earned the highest net favorability marks. Ohio Gov. John Kasich was the only Republican candidate to receive above water favorability scores among the same group, though a large portion had not heard enough about the governor to form an opinion. *score: 0.802007, doc-id: More Republicans in Federal Government.docx*
 7. Democratic feds seem more willing to consider Republican presidential candidates, giving Ohio Gov. John Kasich and former Florida Gov. Jeb Bush higher favorability scores than their party’s own Martin O’Malley and Jim Webb. Meanwhile, just 4 percent of Republican respondents viewed Democratic frontrunner Hillary Clinton favorably. <b>Republicans in the federal workforce are more likely to view national security as the most important issue when choosing a presidential candidate, while Democrats are more likely to consider federal workforce issues.</b> Democratic feds listed economic policy, federal workforce issues and health care as their top three issues, while Republicans listed national security, economic policy and immigration. Among feds from both parties who listed federal workforce issues as very important or essential, Democratic presidential candidate Sen. Bernie Sanders, I-Vt., earned the highest net favorability marks. *score: 0.7985996, doc-id: More Republicans in Federal Government.docx*
 8. Case-Specific Questions - Contact the U.S. Embassy or Consulate handling your visa application for status information. Select U.S. Embassy or Consulate for contact information. <b>General Questions - review Contact Us.</b> *score: 0.76912034, doc-id: US - Visitor Visa.docx*
 9. <b>There Are More Republicans in Federal Government Than You Might Think Four in 10 feds identify with the GOP, poll shows.</b> AUGUST 14, 2015 Eric Katz ERIC KATZ Senior Correspondent Given the proclivity of Republican politicians to deride the federal government and the bureaucrats that fill its ranks, it stands to reason Democrats would be more likely to seek out Uncle Sam for employment. But only barely. In a recent survey conducted by Government Business Council, the research arm of Government Executive Media Group, 44 percent of respondents identified as Democrats or Democratic-leaning independents, while 40 percent identified as Republicans or Republican-leaning independents. *score: 0.7644344, doc-id: More Republicans in Federal Government.docx*
 10. Among feds from both parties who listed federal workforce issues as very important or essential, Democratic presidential candidate Sen. Bernie Sanders, I-Vt., earned the highest net favorability marks. Ohio Gov. John Kasich was the only Republican candidate to receive above water favorability scores among the same group, though a large portion had not heard enough about the governor to form an opinion. <b>Republican frontrunner Donald Trump was viewed very unfavorably by the group overall, but 24 percent of Republicans who prioritize federal workforce issues said they would vote for him if the primary were held today.</b> His lead among that group was even larger than among Republican feds. The poll had 973 federal workers participating, with a 3 percent margin of error. *score: 0.7577404, doc-id: More Republicans in Federal Government.docx*
